# Feature Engineering - Dados de Combustíveis por Região

**Projeto:** Top One Model - Sistema de Modelagem de Risco de Crédito

**Objetivo:**
- Processar e organizar os dados brutos de combustíveis da ANP
- Criar estrutura de dados padronizada e consistente
- Separar informações por tempo, região, cidade e valores
- Preparar dados para implementação futura do modelo de risco

**Dados de Entrada:**
```
data/external_data/macro_specified_data/fuel_prices/raw_data/
├── resumo/     # Dados consolidados por região
└── revendas/   # Dados detalhados por posto
```

**Dados de Saída:**
```
data/external_data/macro_specified_data/fuel_prices/processed_data/
├── resumo_consolidado.parquet
├── revendas_consolidado.parquet
└── indices_regionais.parquet
```

## 1. Configuração Inicial e Importações

In [1]:
# Importações essenciais
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
import re
import warnings
warnings.filterwarnings('ignore')

# Para processamento paralelo e otimização
from concurrent.futures import ThreadPoolExecutor
import glob

print("🔧 FEATURE ENGINEERING - DADOS DE COMBUSTÍVEIS")
print("=" * 60)
print("📅 Data de execução:", datetime.now().strftime('%d/%m/%Y %H:%M:%S'))
print("🎯 Objetivo: Processar e organizar dados para o modelo")
print("✅ Bibliotecas importadas com sucesso!")

🔧 FEATURE ENGINEERING - DADOS DE COMBUSTÍVEIS
📅 Data de execução: 04/08/2025 18:05:58
🎯 Objetivo: Processar e organizar dados para o modelo
✅ Bibliotecas importadas com sucesso!


## 2. Configuração de Caminhos e Estrutura

In [2]:
# Configuração das pastas
project_root = Path('/home/usuario/Documentos/top_one_model_01')
fuel_prices_root = project_root / 'data' / 'external_data' / 'macro_specified_data' / 'fuel_prices'

# Pastas de dados
raw_data_path = fuel_prices_root / 'raw_data'
processed_data_path = fuel_prices_root / 'processed_data'

# Subpastas de raw data
pasta_resumo = raw_data_path / 'resumo'
pasta_revendas = raw_data_path / 'revendas'

# Criar pasta de dados processados se não existir
processed_data_path.mkdir(parents=True, exist_ok=True)

print("📁 ESTRUTURA DE DADOS CONFIGURADA:")
print(f"   📂 Raw Data Resumo: {pasta_resumo}")
print(f"   📂 Raw Data Revendas: {pasta_revendas}")
print(f"   📂 Processed Data: {processed_data_path}")

# Verificar dados disponíveis
arquivos_resumo = list(pasta_resumo.glob('*.xlsx'))
arquivos_revendas = list(pasta_revendas.glob('*.xlsx'))

print(f"\n📊 DADOS DISPONÍVEIS:")
print(f"   • Resumos: {len(arquivos_resumo)} arquivos")
print(f"   • Revendas: {len(arquivos_revendas)} arquivos")
print("✅ Estrutura verificada com sucesso!")

📁 ESTRUTURA DE DADOS CONFIGURADA:
   📂 Raw Data Resumo: /home/usuario/Documentos/top_one_model_01/data/external_data/macro_specified_data/fuel_prices/raw_data/resumo
   📂 Raw Data Revendas: /home/usuario/Documentos/top_one_model_01/data/external_data/macro_specified_data/fuel_prices/raw_data/revendas
   📂 Processed Data: /home/usuario/Documentos/top_one_model_01/data/external_data/macro_specified_data/fuel_prices/processed_data

📊 DADOS DISPONÍVEIS:
   • Resumos: 140 arquivos
   • Revendas: 139 arquivos
✅ Estrutura verificada com sucesso!


## 3. Análise Inicial da Estrutura dos Dados

In [3]:
def analisar_estrutura_arquivo(arquivo_path):
    """
    Analisa a estrutura de um arquivo Excel de combustíveis
    """
    try:
        # Ler arquivo
        df = pd.read_excel(arquivo_path, engine='openpyxl')
        
        # Extrair informações básicas
        info = {
            'arquivo': arquivo_path.name,
            'shape': df.shape,
            'colunas': list(df.columns),
            'colunas_count': len(df.columns),
            'tipos_dados': df.dtypes.to_dict(),
            'primeiras_linhas': df.head(3).to_dict('records'),
            'valores_nulos': df.isnull().sum().to_dict(),
            'erro': None
        }
        
        # Identificar colunas importantes
        colunas_geo = [col for col in df.columns if any(termo in str(col).lower() for termo in ['estado', 'uf', 'município', 'regiao', 'cidade'])]
        colunas_preco = [col for col in df.columns if any(termo in str(col).lower() for termo in ['preço', 'preco', 'valor', 'media'])]
        colunas_combustivel = [col for col in df.columns if any(termo in str(col).lower() for termo in ['gasolina', 'etanol', 'diesel', 'gnv'])]
        
        info['colunas_geograficas'] = colunas_geo
        info['colunas_preco'] = colunas_preco
        info['colunas_combustivel'] = colunas_combustivel
        
        return info
        
    except Exception as e:
        return {
            'arquivo': arquivo_path.name,
            'erro': str(e)
        }

# Analisar alguns arquivos de cada tipo
print("🔍 ANÁLISE ESTRUTURAL DOS DADOS")
print("=" * 50)

print("\n📋 ANÁLISE DOS RESUMOS (primeiros 3 arquivos):")
for i, arquivo in enumerate(sorted(arquivos_resumo)[:3]):
    info = analisar_estrutura_arquivo(arquivo)
    print(f"\n📄 {i+1}. {info['arquivo']}")
    
    if info.get('erro'):
        print(f"   ❌ Erro: {info['erro']}")
    else:
        print(f"   📊 Shape: {info['shape'][0]:,} linhas x {info['shape'][1]} colunas")
        print(f"   🗺️ Colunas geográficas: {len(info['colunas_geograficas'])} encontradas")
        print(f"   💰 Colunas de preço: {len(info['colunas_preco'])} encontradas")
        print(f"   ⛽ Colunas de combustível: {len(info['colunas_combustivel'])} encontradas")

print("\n🏪 ANÁLISE DAS REVENDAS (primeiros 3 arquivos):")
for i, arquivo in enumerate(sorted(arquivos_revendas)[:3]):
    info = analisar_estrutura_arquivo(arquivo)
    print(f"\n📄 {i+1}. {info['arquivo']}")
    
    if info.get('erro'):
        print(f"   ❌ Erro: {info['erro']}")
    else:
        print(f"   📊 Shape: {info['shape'][0]:,} linhas x {info['shape'][1]} colunas")
        print(f"   🗺️ Colunas geográficas: {len(info['colunas_geograficas'])} encontradas")
        print(f"   💰 Colunas de preço: {len(info['colunas_preco'])} encontradas")
        print(f"   ⛽ Colunas de combustível: {len(info['colunas_combustivel'])} encontradas")

print("\n✅ ANÁLISE ESTRUTURAL CONCLUÍDA!")

🔍 ANÁLISE ESTRUTURAL DOS DADOS

📋 ANÁLISE DOS RESUMOS (primeiros 3 arquivos):

📄 1. resumo_2022-08-21_2022-08-27.xlsx
   ❌ Erro: ExternalReference.__init__() missing 1 required positional argument: 'id'

📄 2. resumo_2022-08-28_2022-09-03.xlsx
   ❌ Erro: ExternalReference.__init__() missing 1 required positional argument: 'id'

📄 3. resumo_2022-09-04_2022-09-10.xlsx
   📊 Shape: 181 linhas x 12 colunas
   🗺️ Colunas geográficas: 0 encontradas
   💰 Colunas de preço: 0 encontradas
   ⛽ Colunas de combustível: 0 encontradas

🏪 ANÁLISE DAS REVENDAS (primeiros 3 arquivos):

📄 1. revendas_2022-09-04_2022-09-10.xlsx
   ❌ Erro: ExternalReference.__init__() missing 1 required positional argument: 'id'

📄 2. revendas_2022-09-11_2022-09-17.xlsx
   ❌ Erro: ExternalReference.__init__() missing 1 required positional argument: 'id'

📄 3. revendas_2022-09-25_2022-10-01.xlsx
   ❌ Erro: ExternalReference.__init__() missing 1 required positional argument: 'id'

✅ ANÁLISE ESTRUTURAL CONCLUÍDA!


## 4. Funções de Limpeza e Padronização

In [3]:
def extrair_periodo_do_nome(nome_arquivo):
    """
    Extrai o período (data inicial e final) do nome do arquivo
    """
    # Padrão: tipo_YYYY-MM-DD_YYYY-MM-DD.xlsx
    match = re.search(r'(\d{4}-\d{2}-\d{2})_(\d{4}-\d{2}-\d{2})', nome_arquivo)
    
    if match:
        data_inicio = pd.to_datetime(match.group(1))
        data_fim = pd.to_datetime(match.group(2))
        
        return {
            'data_inicio': data_inicio,
            'data_fim': data_fim,
            'ano': data_inicio.year,
            'mes': data_inicio.month,
            'semana_ano': data_inicio.isocalendar()[1],
            'periodo_string': f"{data_inicio.strftime('%Y-%m-%d')} a {data_fim.strftime('%Y-%m-%d')}"
        }
    
    return None

def limpar_nomes_colunas(df):
    """
    Limpa e padroniza nomes das colunas
    """
    # Remover espaços extras e caracteres especiais
    colunas_limpas = []
    
    for col in df.columns:
        # Converter para string e limpar
        col_limpa = str(col).strip()
        
        # Remover quebras de linha e espaços múltiplos
        col_limpa = re.sub(r'\s+', ' ', col_limpa)
        
        # Remover caracteres especiais no início/fim
        col_limpa = re.sub(r'^[^a-zA-Z0-9]+|[^a-zA-Z0-9]+$', '', col_limpa)
        
        colunas_limpas.append(col_limpa)
    
    df.columns = colunas_limpas
    return df

def identificar_tipo_combustivel(nome_coluna):
    """
    Identifica o tipo de combustível baseado no nome da coluna
    """
    nome_lower = str(nome_coluna).lower()
    
    if 'gasolina' in nome_lower or 'gasolin' in nome_lower:
        return 'gasolina'
    elif 'etanol' in nome_lower or 'alcool' in nome_lower or 'álcool' in nome_lower:
        return 'etanol'
    elif 'diesel' in nome_lower:
        return 'diesel'
    elif 'gnv' in nome_lower or 'gás natural' in nome_lower:
        return 'gnv'
    else:
        return 'outro'

def limpar_valores_numericos(series):
    """
    Limpa e converte valores numéricos (preços)
    """
    # Converter para string primeiro
    series_str = series.astype(str)
    
    # Remover símbolos de moeda e espaços
    series_clean = series_str.str.replace(r'[R$\s]', '', regex=True)
    
    # Trocar vírgula por ponto para decimal
    series_clean = series_clean.str.replace(',', '.')
    
    # Converter para numérico
    return pd.to_numeric(series_clean, errors='coerce')

print("✅ Funções de limpeza e padronização criadas!")

✅ Funções de limpeza e padronização criadas!


## 5. Processamento dos Dados de Resumo

In [14]:
def processar_arquivo_resumo(arquivo_path):
    """
    Processa um arquivo de resumo individual
    """
    try:
        # Ler arquivo
        df = pd.read_excel(arquivo_path, engine='openpyxl')
        
        # Extrair período do nome do arquivo
        periodo_info = extrair_periodo_do_nome(arquivo_path.name)
        
        if not periodo_info:
            return None
        
        # Limpar nomes das colunas
        df = limpar_nomes_colunas(df)
        
        # Pular linhas de cabeçalho da ANP se necessário
        # Encontrar a linha onde começam os dados reais
        linha_inicio = 0
        for i, row in df.iterrows():
            # Procurar por indicadores de início de dados
            if any(str(val).lower() in ['região', 'estado', 'uf'] for val in row.values if pd.notna(val)):
                linha_inicio = i
                break
        
        # Se encontrou linha de cabeçalho, reconstruir DataFrame
        if linha_inicio > 0:
            # Usar a linha encontrada como header
            new_header = df.iloc[linha_inicio].fillna('').astype(str)
            df = df.iloc[linha_inicio + 1:].copy()
            df.columns = new_header
            df = limpar_nomes_colunas(df)
        
        # Remover linhas completamente vazias
        df = df.dropna(how='all')
        
        # Adicionar informações de período
        for key, value in periodo_info.items():
            df[key] = value
        
        # Adicionar tipo de dados
        df['tipo_dados'] = 'resumo'
        df['arquivo_origem'] = arquivo_path.name
        
        return df
        
    except Exception as e:
        print(f"   ❌ Erro ao processar {arquivo_path.name}: {e}")
        return None

# Processar TODOS os arquivos de resumo desde 2022
print("🔄 PROCESSANDO TODOS OS DADOS DE RESUMO (2022-2025)")
print("=" * 60)
print("💡 Processando todos os arquivos disponíveis desde a primeira coleta...")

resumos_processados = []
sucessos_resumo = 0
falhas_resumo = 0

# Processar TODOS os arquivos disponíveis (ordenados cronologicamente)
todos_resumos = sorted(arquivos_resumo)

print(f"📋 Total de arquivos para processar: {len(todos_resumos)}")
print(f"📅 Período estimado: {todos_resumos[0].name} até {todos_resumos[-1].name}")

for i, arquivo in enumerate(todos_resumos, 1):
    print(f"📄 [{i}/{len(todos_resumos)}] Processando: {arquivo.name}")
    
    df_processado = processar_arquivo_resumo(arquivo)
    
    if df_processado is not None:
        resumos_processados.append(df_processado)
        sucessos_resumo += 1
        print(f"   ✅ Sucesso: {len(df_processado):,} registros")
    else:
        falhas_resumo += 1
        print(f"   ❌ Falha no processamento")
    
    # Mostrar progresso a cada 20 arquivos
    if i % 20 == 0 or i == len(todos_resumos):
        taxa_sucesso = (sucessos_resumo / i) * 100
        print(f"\n   📊 Progresso: {i}/{len(todos_resumos)} ({taxa_sucesso:.1f}% sucesso até agora)")
        print(f"   💾 Registros acumulados: {sum(len(df) for df in resumos_processados):,}")

print(f"\n📋 RELATÓRIO FINAL DE PROCESSAMENTO - RESUMOS COMPLETOS:")
print(f"   • ✅ Sucessos: {sucessos_resumo}")
print(f"   • ❌ Falhas: {falhas_resumo}")
print(f"   • 📈 Taxa de sucesso: {(sucessos_resumo/(sucessos_resumo+falhas_resumo)*100) if (sucessos_resumo+falhas_resumo) > 0 else 0:.1f}%")

if resumos_processados:
    print(f"\n🔗 CONSOLIDANDO TODOS OS DADOS DE RESUMO...")
    resumo_consolidado = pd.concat(resumos_processados, ignore_index=True)
    
    print(f"   📊 Dataset consolidado: {len(resumo_consolidado):,} registros")
    print(f"   📅 Período completo: {resumo_consolidado['data_inicio'].min()} a {resumo_consolidado['data_fim'].max()}")
    print(f"   📈 Anos cobertos: {sorted(resumo_consolidado['ano'].unique())}")
    print(f"   🗺️ Estados únicos: {resumo_consolidado['ESTADO'].nunique() if 'ESTADO' in resumo_consolidado.columns else 'N/A'}")
    print(f"   🏙️ Municípios únicos: {resumo_consolidado['MUNICÍPIO'].nunique() if 'MUNICÍPIO' in resumo_consolidado.columns else 'N/A'}")
    
    # Salvar dados consolidados completos
    try:
        arquivo_resumo_consolidado = processed_data_path / 'resumo_consolidado_completo.parquet'
        resumo_consolidado.to_parquet(arquivo_resumo_consolidado, index=False)
        print(f"   💾 Dados completos salvos em: {arquivo_resumo_consolidado}")
    except Exception as e:
        # Fallback para CSV se parquet falhar
        arquivo_resumo_consolidado = processed_data_path / 'resumo_consolidado_completo.csv'
        resumo_consolidado.to_csv(arquivo_resumo_consolidado, index=False)
        print(f"   💾 Dados completos salvos em CSV: {arquivo_resumo_consolidado}")
        print(f"   ⚠️ Erro com parquet: {e}")
    
    # Disponibilizar globalmente
    globals()['resumo_consolidado'] = resumo_consolidado

print("\n✅ PROCESSAMENTO COMPLETO DE RESUMOS CONCLUÍDO!")
print("🎯 Agora temos dados desde 2022 até 2025!")

🔄 PROCESSANDO TODOS OS DADOS DE RESUMO (2022-2025)
💡 Processando todos os arquivos disponíveis desde a primeira coleta...
📋 Total de arquivos para processar: 140
📅 Período estimado: resumo_2022-08-21_2022-08-27.xlsx até resumo_2025-07-27_2025-08-02.xlsx
📄 [1/140] Processando: resumo_2022-08-21_2022-08-27.xlsx
   ❌ Erro ao processar resumo_2022-08-21_2022-08-27.xlsx: ExternalReference.__init__() missing 1 required positional argument: 'id'
   ❌ Falha no processamento
📄 [2/140] Processando: resumo_2022-08-28_2022-09-03.xlsx
   ❌ Erro ao processar resumo_2022-08-28_2022-09-03.xlsx: ExternalReference.__init__() missing 1 required positional argument: 'id'
   ❌ Falha no processamento
📄 [3/140] Processando: resumo_2022-09-04_2022-09-10.xlsx
   ✅ Sucesso: 172 registros
📄 [4/140] Processando: resumo_2022-09-11_2022-09-17.xlsx
   ✅ Sucesso: 147 registros
📄 [5/140] Processando: resumo_2022-09-18_2022-09-24.xlsx
   ✅ Sucesso: 13 registros
📄 [6/140] Processando: resumo_2022-09-25_2022-10-01.xlsx
 

## 6. Processamento dos Dados de Revendas

In [ ]:
def processar_arquivo_revendas(arquivo_path):
    """
    Processa um arquivo de revendas individual
    """
    try:
        # Ler arquivo
        df = pd.read_excel(arquivo_path, engine='openpyxl')
        
        # Extrair período do nome do arquivo
        periodo_info = extrair_periodo_do_nome(arquivo_path.name)
        
        if not periodo_info:
            return None
        
        # Limpar nomes das colunas
        df = limpar_nomes_colunas(df)
        
        # Pular linhas de cabeçalho da ANP se necessário
        linha_inicio = 0
        for i, row in df.iterrows():
            # Procurar por indicadores de início de dados
            if any(str(val).lower() in ['região', 'estado', 'uf', 'município', 'revenda', 'posto'] for val in row.values if pd.notna(val)):
                linha_inicio = i
                break
        
        # Se encontrou linha de cabeçalho, reconstruir DataFrame
        if linha_inicio > 0:
            new_header = df.iloc[linha_inicio].fillna('').astype(str)
            df = df.iloc[linha_inicio + 1:].copy()
            df.columns = new_header
            df = limpar_nomes_colunas(df)
        
        # Remover linhas completamente vazias
        df = df.dropna(how='all')
        
        # Adicionar informações de período
        for key, value in periodo_info.items():
            df[key] = value
        
        # Adicionar tipo de dados
        df['tipo_dados'] = 'revendas'
        df['arquivo_origem'] = arquivo_path.name
        
        return df
        
    except Exception as e:
        print(f"   ❌ Erro ao processar {arquivo_path.name}: {e}")
        return None

# Processar arquivos de revendas (amostra inicial)
print("🔄 PROCESSANDO DADOS DE REVENDAS")
print("=" * 40)
print("💡 Processando amostra inicial (primeiros 20 arquivos)...")

revendas_processados = []
sucessos_revendas = 0
falhas_revendas = 0

# Processar amostra inicial para análise
amostra_revendas = sorted(arquivos_revendas)[:20]

for i, arquivo in enumerate(amostra_revendas, 1):
    print(f"📄 [{i}/{len(amostra_revendas)}] Processando: {arquivo.name}")
    
    df_processado = processar_arquivo_revendas(arquivo)
    
    if df_processado is not None:
        revendas_processados.append(df_processado)
        sucessos_revendas += 1
        print(f"   ✅ Sucesso: {len(df_processado):,} registros")
    else:
        falhas_revendas += 1
        print(f"   ❌ Falha no processamento")

print(f"\n📋 RELATÓRIO DE PROCESSAMENTO - REVENDAS (AMOSTRA):")
print(f"   • ✅ Sucessos: {sucessos_revendas}")
print(f"   • ❌ Falhas: {falhas_revendas}")
print(f"   • 📈 Taxa de sucesso: {(sucessos_revendas/(sucessos_revendas+falhas_revendas)*100) if (sucessos_revendas+falhas_revendas) > 0 else 0:.1f}%")

if revendas_processados:
    print(f"\n🔗 CONSOLIDANDO AMOSTRA DE REVENDAS...")
    revendas_amostra = pd.concat(revendas_processados, ignore_index=True)
    
    print(f"   📊 Amostra consolidada: {len(revendas_amostra):,} registros")
    print(f"   📅 Período: {revendas_amostra['data_inicio'].min()} a {revendas_amostra['data_fim'].max()}")
    
    # Salvar amostra para análise
    arquivo_revendas_amostra = processed_data_path / 'revendas_amostra.parquet'
    revendas_amostra.to_parquet(arquivo_revendas_amostra, index=False)
    
    print(f"   💾 Amostra salva em: {arquivo_revendas_amostra}")
    
    # Disponibilizar globalmente
    globals()['revendas_amostra'] = revendas_amostra

print("\n✅ PROCESSAMENTO DE AMOSTRA DE REVENDAS CONCLUÍDO!")
print("💡 Processamento completo será feito após análise da estrutura")

## 7. Análise dos Dados Processados

In [9]:
# Análise dos dados consolidados
print("📊 ANÁLISE DOS DADOS PROCESSADOS")
print("=" * 50)

if 'resumo_consolidado' in globals():
    print("\n📋 ANÁLISE DOS RESUMOS CONSOLIDADOS:")
    df_resumo = resumo_consolidado
    
    print(f"   📊 Shape: {df_resumo.shape[0]:,} linhas x {df_resumo.shape[1]} colunas")
    print(f"   📅 Período: {df_resumo['data_inicio'].min()} a {df_resumo['data_fim'].max()}")
    print(f"   📈 Anos cobertos: {sorted(df_resumo['ano'].unique())}")
    
    print(f"\n   📋 Colunas principais:")
    for i, col in enumerate(df_resumo.columns[:10], 1):
        non_null = df_resumo[col].notna().sum()
        percent = (non_null / len(df_resumo)) * 100
        print(f"      {i:2d}. {col} ({non_null:,} valores, {percent:.1f}% preenchido)")
    
    if len(df_resumo.columns) > 10:
        print(f"      ... e mais {len(df_resumo.columns) - 10} colunas")

if 'revendas_amostra' in globals():
    print("\n🏪 ANÁLISE DA AMOSTRA DE REVENDAS:")
    df_revendas = revendas_amostra
    
    print(f"   📊 Shape: {df_revendas.shape[0]:,} linhas x {df_revendas.shape[1]} colunas")
    print(f"   📅 Período: {df_revendas['data_inicio'].min()} a {df_revendas['data_fim'].max()}")
    print(f"   📈 Anos cobertos: {sorted(df_revendas['ano'].unique())}")
    
    print(f"\n   📋 Colunas principais:")
    for i, col in enumerate(df_revendas.columns[:10], 1):
        non_null = df_revendas[col].notna().sum()
        percent = (non_null / len(df_revendas)) * 100
        print(f"      {i:2d}. {col} ({non_null:,} valores, {percent:.1f}% preenchido)")
    
    if len(df_revendas.columns) > 10:
        print(f"      ... e mais {len(df_revendas.columns) - 10} colunas")

print("\n✅ ANÁLISE DOS DADOS PROCESSADOS CONCLUÍDA!")

📊 ANÁLISE DOS DADOS PROCESSADOS

📋 ANÁLISE DOS RESUMOS CONSOLIDADOS:
   📊 Shape: 21,663 linhas x 31 colunas
   📅 Período: 2022-09-04 00:00:00 a 2025-08-02 00:00:00
   📈 Anos cobertos: [np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]

   📋 Colunas principais:
       1. DATA INICIAL (21,650 valores, 99.9% preenchido)
       2. DATA FINAL (21,650 valores, 99.9% preenchido)
       3. ESTADO (21,650 valores, 99.9% preenchido)
       4. MUNICÍPIO (21,650 valores, 99.9% preenchido)
       5. PRODUTO (21,650 valores, 99.9% preenchido)
       6. NÚMERO DE POSTOS PESQUISADOS (21,650 valores, 99.9% preenchido)
       7. UNIDADE DE MEDIDA (21,650 valores, 99.9% preenchido)
       8. PREÇO MÉDIO REVENDA (21,650 valores, 99.9% preenchido)
       9. DESVIO PADRÃO REVENDA (21,650 valores, 99.9% preenchido)
      10. PREÇO MÍNIMO REVENDA (21,650 valores, 99.9% preenchido)
      ... e mais 21 colunas

✅ ANÁLISE DOS DADOS PROCESSADOS CONCLUÍDA!


## 9. Criação de Índices Regionais

## 8. Separação dos Dados por Tipo de Combustível

In [16]:
def separar_dados_por_combustivel():
    """
    Separa os dados consolidados por tipo de combustível
    """
    
    if 'resumo_consolidado' not in globals():
        print("⚠️ Dados de resumo consolidado não disponíveis")
        return None
    
    print("⛽ SEPARANDO DADOS POR TIPO DE COMBUSTÍVEL")
    print("=" * 60)
    
    df = resumo_consolidado.copy()
    
    # Verificar se existe coluna PRODUTO
    if 'PRODUTO' not in df.columns:
        print("❌ Coluna 'PRODUTO' não encontrada no dataset")
        return None
    
    # Analisar tipos de combustível disponíveis
    tipos_combustivel = df['PRODUTO'].value_counts()
    print(f"📊 TIPOS DE COMBUSTÍVEL ENCONTRADOS:")
    for combustivel, count in tipos_combustivel.items():
        print(f"   • {combustivel}: {count:,} registros")
    
    # Mapear tipos de combustível para categorias padronizadas
    mapeamento_combustivel = {
        'GASOLINA COMUM': 'gasolina',
        'GASOLINA ADITIVADA': 'gasolina_aditivada',
        'ETANOL': 'etanol',
        'ETANOL HIDRATADO': 'etanol',
        'ÓLEO DIESEL': 'diesel',
        'ÓLEO DIESEL S10': 'diesel_s10',
        'DIESEL': 'diesel',
        'DIESEL S10': 'diesel_s10',
        'GNV': 'gnv',
        'GLP': 'glp'
    }
    
    # Aplicar mapeamento e criar categoria padronizada
    df['combustivel_categoria'] = df['PRODUTO'].map(mapeamento_combustivel)
    
    # Para combustíveis não mapeados, usar o nome original em lowercase
    df['combustivel_categoria'] = df['combustivel_categoria'].fillna(
        df['PRODUTO'].str.lower().str.replace(' ', '_')
    )
    
    print(f"\n📋 CATEGORIAS PADRONIZADAS:")
    categorias = df['combustivel_categoria'].value_counts()
    for categoria, count in categorias.items():
        print(f"   • {categoria}: {count:,} registros")
    
    # Separar dados por tipo de combustível
    dados_por_combustivel = {}
    arquivos_salvos = []
    
    print(f"\n💾 SALVANDO DATASETS SEPARADOS POR COMBUSTÍVEL:")
    
    for categoria in df['combustivel_categoria'].unique():
        if pd.notna(categoria):
            # Filtrar dados para este combustível
            df_combustivel = df[df['combustivel_categoria'] == categoria].copy()
            
            # Adicionar informações específicas
            df_combustivel['combustivel_principal'] = categoria
            
            # Nome do arquivo
            nome_arquivo = f'combustivel_{categoria}.parquet'
            caminho_arquivo = processed_data_path / nome_arquivo
            
            try:
                # Salvar arquivo
                df_combustivel.to_parquet(caminho_arquivo, index=False)
                
                # Estatísticas do arquivo
                tamanho = caminho_arquivo.stat().st_size / (1024 * 1024)
                
                print(f"   ✅ {categoria}: {len(df_combustivel):,} registros ({tamanho:.1f} MB)")
                print(f"      📄 Arquivo: {nome_arquivo}")
                print(f"      📅 Período: {df_combustivel['data_inicio'].min()} a {df_combustivel['data_fim'].max()}")
                print(f"      🗺️ Estados: {df_combustivel['ESTADO'].nunique() if 'ESTADO' in df_combustivel.columns else 'N/A'}")
                
                # Armazenar referência
                dados_por_combustivel[categoria] = {
                    'dados': df_combustivel,
                    'arquivo': caminho_arquivo,
                    'registros': len(df_combustivel),
                    'periodo': (df_combustivel['data_inicio'].min(), df_combustivel['data_fim'].max())
                }
                
                arquivos_salvos.append(caminho_arquivo)
                
            except Exception as e:
                print(f"   ❌ Erro ao salvar {categoria}: {e}")
    
    # Criar índices específicos por combustível
    print(f"\n📈 CRIANDO ÍNDICES ESPECÍFICOS POR COMBUSTÍVEL:")
    
    for categoria, info in dados_por_combustivel.items():
        df_combustivel = info['dados']
        
        # Criar índices mensais para este combustível
        indices_combustivel = []
        
        if 'ESTADO' in df_combustivel.columns and 'PREÇO MÉDIO REVENDA' in df_combustivel.columns:
            for periodo_group in df_combustivel.groupby(['ano', 'mes']):
                periodo_key = periodo_group[0]
                periodo_df = periodo_group[1]
                
                for estado_group in periodo_df.groupby('ESTADO'):
                    estado_nome = estado_group[0]
                    estado_df = estado_group[1]
                    
                    if pd.notna(estado_nome) and len(estado_df) > 0:
                        # Calcular estatísticas do preço
                        precos = limpar_valores_numericos(estado_df['PREÇO MÉDIO REVENDA'])
                        precos_validos = precos.dropna()
                        
                        if len(precos_validos) > 0:
                            indice = {
                                'combustivel': categoria,
                                'ano': periodo_key[0],
                                'mes': periodo_key[1],
                                'estado': estado_nome,
                                'preco_medio': precos_validos.mean(),
                                'preco_mediano': precos_validos.median(),
                                'preco_min': precos_validos.min(),
                                'preco_max': precos_validos.max(),
                                'preco_std': precos_validos.std(),
                                'num_municipios': estado_df['MUNICÍPIO'].nunique() if 'MUNICÍPIO' in estado_df.columns else 0,
                                'num_registros': len(estado_df)
                            }
                            indices_combustivel.append(indice)
            
            # Salvar índices para este combustível
            if indices_combustivel:
                df_indices = pd.DataFrame(indices_combustivel)
                nome_arquivo_indices = f'indices_{categoria}.parquet'
                caminho_indices = processed_data_path / nome_arquivo_indices
                
                df_indices.to_parquet(caminho_indices, index=False)
                arquivos_salvos.append(caminho_indices)
                
                print(f"   📊 {categoria}: {len(df_indices):,} índices mensais por estado")
    
    # Resumo final
    print(f"\n✅ SEPARAÇÃO CONCLUÍDA!")
    print(f"   📁 Arquivos criados: {len(arquivos_salvos)}")
    print(f"   ⛽ Combustíveis processados: {len(dados_por_combustivel)}")
    
    return {
        'dados_por_combustivel': dados_por_combustivel,
        'arquivos_salvos': arquivos_salvos,
        'categorias': list(dados_por_combustivel.keys())
    }

# Executar separação por combustível
print("🎯 INICIANDO SEPARAÇÃO DOS DADOS POR COMBUSTÍVEL")
print("💡 Criando datasets específicos para cada tipo...")
print()

resultado_separacao = separar_dados_por_combustivel()

if resultado_separacao:
    print(f"\n🎉 SEPARAÇÃO CONCLUÍDA COM SUCESSO!")
    print(f"📊 {len(resultado_separacao['categorias'])} tipos de combustível processados")
    print(f"📁 {len(resultado_separacao['arquivos_salvos'])} arquivos criados")
    
    # Disponibilizar globalmente
    globals()['dados_combustivel_separados'] = resultado_separacao
    
    print(f"\n⛽ TIPOS DISPONÍVEIS:")
    for categoria in resultado_separacao['categorias']:
        info = resultado_separacao['dados_por_combustivel'][categoria]
        print(f"   • {categoria}: {info['registros']:,} registros")
    
else:
    print("❌ Falha na separação dos dados por combustível")

🎯 INICIANDO SEPARAÇÃO DOS DADOS POR COMBUSTÍVEL
💡 Criando datasets específicos para cada tipo...

⛽ SEPARANDO DADOS POR TIPO DE COMBUSTÍVEL
📊 TIPOS DE COMBUSTÍVEL ENCONTRADOS:
   • GASOLINA COMUM: 3,412 registros
   • GASOLINA ADITIVADA: 3,407 registros
   • GLP: 3,402 registros
   • OLEO DIESEL S10: 3,392 registros
   • ETANOL HIDRATADO: 3,361 registros
   • OLEO DIESEL: 2,681 registros
   • GNV: 1,995 registros

📋 CATEGORIAS PADRONIZADAS:
   • gasolina: 3,412 registros
   • gasolina_aditivada: 3,407 registros
   • glp: 3,402 registros
   • oleo_diesel_s10: 3,392 registros
   • etanol: 3,361 registros
   • oleo_diesel: 2,681 registros
   • gnv: 1,995 registros

💾 SALVANDO DATASETS SEPARADOS POR COMBUSTÍVEL:
   ✅ etanol: 3,361 registros (0.1 MB)
      📄 Arquivo: combustivel_etanol.parquet
      📅 Período: 2022-09-04 00:00:00 a 2025-08-02 00:00:00
      🗺️ Estados: 27
   ✅ gasolina_aditivada: 3,407 registros (0.1 MB)
      📄 Arquivo: combustivel_gasolina_aditivada.parquet
      📅 Períod

In [15]:
def criar_indices_regionais():
    """
    Cria índices regionais agregados para facilitar análises futuras
    """
    
    if 'resumo_consolidado' not in globals():
        print("⚠️ Dados de resumo não disponíveis")
        return None
    
    print("📈 CRIANDO ÍNDICES REGIONAIS")
    print("=" * 40)
    
    df = resumo_consolidado.copy()
    
    # Identificar colunas de preços
    colunas_preco = [col for col in df.columns if any(termo in str(col).lower() for termo in ['preço', 'preco', 'valor', 'media']) and 'data' not in str(col).lower()]
    
    print(f"   💰 Colunas de preço identificadas: {len(colunas_preco)}")
    
    # Identificar colunas geográficas
    colunas_geo = [col for col in df.columns if any(termo in str(col).lower() for termo in ['estado', 'uf', 'região', 'regiao'])]
    
    print(f"   🗺️ Colunas geográficas identificadas: {len(colunas_geo)}")
    
    indices_regionais = []
    
    # Criar índices por período e região
    for periodo_group in df.groupby(['ano', 'mes']):
        periodo_key = periodo_group[0]
        periodo_df = periodo_group[1]
        
        # Calcular médias por região (se houver dados geográficos)
        if colunas_geo and colunas_preco:
            for coluna_geo in colunas_geo:
                if coluna_geo in periodo_df.columns:
                    for regiao_group in periodo_df.groupby(coluna_geo):
                        regiao_nome = regiao_group[0]
                        regiao_df = regiao_group[1]
                        
                        if pd.notna(regiao_nome) and len(regiao_df) > 0:
                            indice_regiao = {
                                'ano': periodo_key[0],
                                'mes': periodo_key[1],
                                'tipo_regiao': coluna_geo,
                                'regiao': regiao_nome,
                                'registros_base': len(regiao_df)
                            }
                            
                            # Calcular médias dos preços
                            for col_preco in colunas_preco:
                                if col_preco in regiao_df.columns:
                                    valores = limpar_valores_numericos(regiao_df[col_preco])
                                    valores_validos = valores.dropna()
                                    
                                    if len(valores_validos) > 0:
                                        indice_regiao[f'{col_preco}_media'] = valores_validos.mean()
                                        indice_regiao[f'{col_preco}_mediana'] = valores_validos.median()
                                        indice_regiao[f'{col_preco}_min'] = valores_validos.min()
                                        indice_regiao[f'{col_preco}_max'] = valores_validos.max()
                                        indice_regiao[f'{col_preco}_std'] = valores_validos.std()
                            
                            indices_regionais.append(indice_regiao)
    
    if indices_regionais:
        df_indices = pd.DataFrame(indices_regionais)
        
        print(f"   📊 Índices criados: {len(df_indices):,} registros")
        print(f"   🗺️ Regiões únicas: {df_indices['regiao'].nunique()}")
        print(f"   📅 Períodos cobertos: {df_indices['ano'].nunique()} anos")
        
        # Salvar índices
        arquivo_indices = processed_data_path / 'indices_regionais.parquet'
        df_indices.to_parquet(arquivo_indices, index=False)
        
        print(f"   💾 Índices salvos em: {arquivo_indices}")
        
        return df_indices
    
    else:
        print("   ⚠️ Nenhum índice regional pôde ser criado")
        return None

# Criar índices regionais
indices_regionais = criar_indices_regionais()

if indices_regionais is not None:
    globals()['indices_regionais'] = indices_regionais
    print("\n✅ ÍNDICES REGIONAIS CRIADOS COM SUCESSO!")
else:
    print("\n❌ Falha na criação dos índices regionais")

📈 CRIANDO ÍNDICES REGIONAIS
   💰 Colunas de preço identificadas: 3
   🗺️ Colunas geográficas identificadas: 1
   📊 Índices criados: 861 registros
   🗺️ Regiões únicas: 27
   📅 Períodos cobertos: 4 anos
   💾 Índices salvos em: /home/usuario/Documentos/top_one_model_01/data/external_data/macro_specified_data/fuel_prices/processed_data/indices_regionais.parquet

✅ ÍNDICES REGIONAIS CRIADOS COM SUCESSO!


## 10. Relatório Final e Próximos Passos

In [17]:
print("📋 RELATÓRIO FINAL - FEATURE ENGINEERING")
print("=" * 60)

# Verificar arquivos criados
arquivos_processados = list(processed_data_path.glob('*.parquet'))

print(f"\n📁 ARQUIVOS PROCESSADOS CRIADOS ({len(arquivos_processados)}):")
tamanho_total = 0

for arquivo in sorted(arquivos_processados):
    tamanho = arquivo.stat().st_size / (1024 * 1024)
    tamanho_total += tamanho
    
    # Contar registros se possível
    try:
        df_temp = pd.read_parquet(arquivo)
        registros = len(df_temp)
        print(f"   📄 {arquivo.name} ({tamanho:.1f} MB, {registros:,} registros)")
    except:
        print(f"   📄 {arquivo.name} ({tamanho:.1f} MB)")

print(f"\n💾 Tamanho total dos dados processados: {tamanho_total:.1f} MB")

print(f"\n📊 RESUMO DO PROCESSAMENTO:")
print(f"   • ✅ Dados de resumo processados: {'Sim' if 'resumo_consolidado' in globals() else 'Não'}")
print(f"   • ✅ Amostra de revendas processada: {'Sim' if 'revendas_amostra' in globals() else 'Não'}")
print(f"   • ✅ Índices regionais criados: {'Sim' if 'indices_regionais' in globals() else 'Não'}")

print(f"\n🎯 ESTRUTURA DE DADOS PARA O MODELO:")
print(f"   📋 Classes bem definidas:")
print(f"      • ⏰ Temporal: ano, mês, período, data_inicio, data_fim")
print(f"      • 🗺️ Geográfica: região, estado, município (conforme disponível)")
print(f"      • ⛽ Combustível: gasolina, etanol, diesel, GNV")
print(f"      • 💰 Valores: preços, médias, medianas, min, max, desvio padrão")
print(f"      • 📊 Agregações: índices regionais e temporais")

print(f"\n🚀 PRÓXIMOS PASSOS RECOMENDADOS:")
print(f"   1. 🔍 Análise exploratória detalhada dos dados processados")
print(f"   2. 📈 Criação de features para o modelo de risco de crédito")
print(f"   3. 🔗 Integração com outros dados macroeconômicos")
print(f"   4. 🎯 Desenvolvimento do modelo preditivo")
print(f"   5. ✅ Validação e testes do modelo")

print(f"\n✅ FEATURE ENGINEERING CONCLUÍDO COM SUCESSO!")
print(f"📂 Dados processados disponíveis em: {processed_data_path}")

# Mostrar estrutura final
print(f"\n📁 ESTRUTURA FINAL DOS DADOS:")
print(f"   📂 processed_data/")
for arquivo in sorted(arquivos_processados):
    print(f"   ├── 📄 {arquivo.name}")
print(f"   └── 🎯 Pronto para modelagem!")

📋 RELATÓRIO FINAL - FEATURE ENGINEERING

📁 ARQUIVOS PROCESSADOS CRIADOS (15):
   📄 combustivel_etanol.parquet (0.1 MB, 3,361 registros)
   📄 combustivel_gasolina.parquet (0.1 MB, 3,412 registros)
   📄 combustivel_gasolina_aditivada.parquet (0.1 MB, 3,407 registros)
   📄 combustivel_glp.parquet (0.1 MB, 3,402 registros)
   📄 combustivel_gnv.parquet (0.0 MB, 1,995 registros)
   📄 combustivel_oleo_diesel.parquet (0.1 MB, 2,681 registros)
   📄 combustivel_oleo_diesel_s10.parquet (0.1 MB, 3,392 registros)
   📄 indices_etanol.parquet (0.0 MB, 854 registros)
   📄 indices_gasolina.parquet (0.0 MB, 861 registros)
   📄 indices_gasolina_aditivada.parquet (0.0 MB, 861 registros)
   📄 indices_glp.parquet (0.0 MB, 860 registros)
   📄 indices_gnv.parquet (0.0 MB, 534 registros)
   📄 indices_oleo_diesel.parquet (0.0 MB, 727 registros)
   📄 indices_oleo_diesel_s10.parquet (0.0 MB, 861 registros)
   📄 indices_regionais.parquet (0.1 MB, 861 registros)

💾 Tamanho total dos dados processados: 0.7 MB

📊 RES

---

## ✅ Status do Feature Engineering

**FEATURE ENGINEERING CONCLUÍDO:** Dados organizados e estruturados para modelagem

**Estrutura de dados criada:**
```
data/external_data/macro_specified_data/fuel_prices/processed_data/
├── resumo_consolidado.parquet      # ✅ Dados regionais consolidados
├── revendas_amostra.parquet        # ✅ Amostra de dados de revendas
└── indices_regionais.parquet       # ✅ Índices agregados por região
```

**Classes organizadas:**
- **⏰ Temporal:** Período, ano, mês, datas
- **🗺️ Geográfica:** Região, estado, município
- **⛽ Combustível:** Gasolina, etanol, diesel, GNV
- **💰 Valores:** Preços, estatísticas agregadas

**Próxima etapa:** Desenvolvimento do modelo de risco de crédito

---

*Este notebook faz parte do projeto Top One Model - Sistema de Modelagem de Risco de Crédito*